In [ ]:
import os 

from tensorflow.keras import layers 
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3),
    include_top=False,       # specifying that you want to ignore the fully connected layer at top, and instead go straight to convolutions
    weights=None             # don't want to use the built in weights, but instead want to use snapshot of downloaded model
)

pre_trained_model.load_weights(local_weights_file)

In [ ]:
# go through layers and lock them from training 
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
pre_trained_model.summary()